In [1]:
import csv
from parsel.selector import Selector
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
import pandas as pd

In [3]:
file = open('company_details.csv', 'w+', encoding='utf-8-sig', newline='')
writer = csv.writer(file)
writer.writerow(['Name', 'Type', 'Industry', 'Founded', 'Founders', 'HQ', 'Area served', 'Products/Services', 'Revenue(US$)', 'No: of employees'])

86

In [4]:
driver = webdriver.Firefox()

In [5]:
items = ['Boeing', 'Costco', 'Starbucks', 'Paccar', 'Wells Fargo', 'Zones LLC']

In [6]:
check_list = ['Type', 'Industry', 'Founded', 'Founders', 'Headquarters', 'Area served', 'Products', 'Services', 'Revenue', 'Number of employees']

In [7]:
check_head = check_list = [['Type'], ['Industry'], ['Founded'], ['Founders', 'Founder'], ['Headquarters'], ['Area served'], ['Products', 'Services'], ['Revenue'], ['Number of employees']]

In [8]:
check_sub_list1 = ['Type', 'Industry']
check_sub_list2 = ['Founded', 'Area served', 'Revenue', 'Number of employees']
check_sub_list3 = ['Founders']
check_sub_list4 = ['Headquarters']
check_sub_list5 = ['Products', 'Services']

In [10]:
for item in items:
    
    driver.get('https://en.wikipedia.org/wiki/'+item)
    sleep(2.5)
    
    print('getting information from '+item+'............')
    
    sel = Selector(text = driver.page_source)
    
    check_content = info = sel.xpath('//*[@id = "mw-content-text"]/div/table/tbody/tr/td/b/text()').extract_first()
    if(check_content == 'Wikipedia does not have an article with this exact name.'):
        print('Information from {} is not available on wikipedia'.format(item))
        continue
    
    info = sel.xpath('//*[@class = "infobox vcard"]')
    name = info.xpath('//*[@class = "fn org"]/text()').extract_first()
    info_list = [name]
    
    main_cnt = info.xpath('//tbody').extract_first().count('<tr>')
    
    idx = 0
    
    for i in range(1, main_cnt+1):
        
        
        basic = info.xpath('//tbody/tr[{}]'.format(i))
        
        heading = basic.xpath('th/a/text()').extract_first()
        if(heading not in check_head[idx]):
            heading = basic.xpath('th/text()').extract_first()
        if(heading not in check_head[idx]):
            heading = basic.xpath('th/div/a/text()').extract_first()
        if(heading not in check_head[idx]):
            heading = basic.xpath('th/div/text()').extract_first()
        if(heading not in check_head[idx]):
            continue
            
        if(heading in check_head[idx]):
            if(idx <len(check_head)-1):
                idx+=1
            
#         print(heading)
    
        
        if(heading in check_sub_list1 or heading == 'Founder'):
            info_list.append(basic.xpath('td/a/text()').extract_first())
#             print(info_list, end=' ')
#             print(i)
            
        elif(heading in check_sub_list2):
            temp = basic.xpath('td/text()').extract_first()
            if(heading == 'Revenue'):
                temp = basic.xpath('td/text()')[1].extract()
            info_list.append(temp)
#             print(info_list, end=' ')
#             print(i)
            
        elif(heading in check_sub_list3):
            cnt = basic.xpath('td/div/ul').extract_first().count('<li>')
            temp = 'td/div/ul/'
            founders = ''
            for i in range(1, cnt+1):
                founders+= basic.xpath(temp+'li[{}]/a/text()'.format(i)).extract_first()+', '
            founders = founders[:-2]
            info_list.append(founders)
#             print(info_list, end=' ')
#             print(i)
        
        elif(heading in check_sub_list4):
            area = basic.xpath('td/text()').extract_first()
            city = basic.xpath('td/div/text()').extract_first()
            location = area + city
            info_list.append(location)
#             print(info_list, end=' ')
#             print(i)
            
        elif(heading in check_sub_list5):
            try:
                cnt = basic.xpath('td/div/ul').extract_first().count('<li>')
            except AttributeError:
                cnt = 100
            temp = 'td/div/ul/'
            products = ''
            
            if(cnt > 20):
                cnt = basic.xpath('td').extract_first().count('</a>')
                temp = 'td/'
                for i in range(1, cnt+1):
                    flag = basic.xpath(temp+'a[{}]/text()'.format(i)).extract_first()
                    if(flag == None):
                        flag = basic.xpath(temp+'a[{}]/text()'.format(i)).extract_first()
                    products+= flag+', '
                products = products[:-2]
                info_list.append(products)
            
            
            else:
                for i in range(1, cnt+1):
                    flag = basic.xpath(temp+'li[{}]/text()'.format(i)).extract_first()
                    if(flag == None):
                        flag = basic.xpath(temp+'li[{}]/a/text()'.format(i)).extract_first()
                    products+= flag+', '
                products = products[:-2]
                info_list.append(products)
#             print(info_list, end=' ')
#             print(i)
            
        else:
            pass
            
    
    print('parsed information from '+ item)
    print('')
    sleep(2)
    writer.writerow(info_list)


getting information from Boeing
parsed information from Boeing
getting information from Costco
parsed information from Costco
getting information from Starbucks
parsed information from Starbucks
getting information from Paccar
parsed information from Paccar
getting information from Wells Fargo
parsed information from Wells Fargo
getting information from Zones LLC
Information from Zones LLC is not available on wikipedia


In [11]:
driver.quit()

In [12]:
file.close()

In [13]:
df = pd.read_csv('company_details.csv')

In [14]:
df.head()

,Name,Type,Industry,Founded,Founders,HQ,Area served,Products,Revenue,No: of employees
0,The Boeing Company,Public,South Carolina,"July 15, 1916",William E. Boeing,", ,",Worldwide,"Commercial aircraft, Military aircraft, Satell...",2019\n,"161,133 (January 1, 2020)"
1,Costco Wholesale Corporation,Public,Retail,"July 12, 1976","Amazon Go, Whole Foods",", United States","United States, Canada, United Kingdom, Austral...","Merchandise, Cash & Carry, Warehouse club",(2019),"254,000"
2,Starbucks Corporation,Public,Coffee shop,"March 31, 1971","Jerry Baldwin, Zev Siegl, Gordon Bowker","2401 Utah Avenue South, Seattle, Washington",Worldwide,"Coffee beverages, smoothies, tea, baked goods,...",24.71 billion (2018),"291,000 (2018)"
3,PACCAR Inc,Public,Automotive,1905,NaN,"PACCAR Building, \n","Worldwide (Except Iran, Cuba, Syria, Sudan, an...","Commercial vehicles, , Industrial winches",,"~27,000"
4,Wells Fargo & Company,NaN,Public,18 March 1852,Henry Wells,", \n",NaN,"Asset management, Brokerage services, Banking,...",85.063 billion,"258,700 (2019)"
